In [1]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer/equation_modeling


In [2]:
import pandas as pd

In [3]:
cd ..

/Users/johanbarreiro/Documents/GitHub/vl_optimizer


In [4]:
df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-05T01-00-40_industrial_site2_processed.csv')

# Data Center

[Server Source](https://cc-techgroup.com/data-center-energy-consumption/#:~:text=this%20work%20efficiently.-,How%20Many%20kWh%20Does%20a%20Server%20Use%3F,to%201%2C900%20kWh%20every%20year.)

Average annual power may be around 1,800 to 1,900 kWh every year per server (divide by 365 then 24 the 4)   0.05279680365 kWh per 15 minutes

[Network Switch Source](https://www.genuinemodules.com/how-much-power-does-a-24-port-switch-use_a8539#:~:text=However%2C%20as%20a%20general%20estimate,requirements%20of%20the%20connected%20devices.)


However, as a general estimate, a non-PoE 24 port switch may consume around 20-50 watts of power, while a PoE-enabled switch may consume around 100-250 watts or more, depending on the power requirements of the connected devices. 
PoE: 25Wh - 62.5Wh per 15 minutes
Non-PoE: 5Wh - 12.5Wh per 15 minutes
    
 [HDD & SDD Source](https://dataspan.com/blog/how-much-energy-do-data-centers-use/#:~:text=Depending%20on%20disk%20size%2C%20an,around%206%20watts%20per%20disk.)   

Depending on disk size, an HDD can use anywhere from 6 to 9 watts at maximum capacity. While this is lower than in previous years, power requirements for SSDs have remained mostly constant at around 6 watts per disk.

HDD: 0.175Wh - 0.2625Wh per 15 minutes
SSD: 0.175Wh per 15 minutes


In [5]:
 df.columns

Index(['Time', '02 Production_Electric_Active Energy (kWh)',
       '02 Production_Electric_Active Power (kW)',
       '02 Production_Electric_Apparent Power (kVa)',
       '02 Production_Electric_Power Factor (real)',
       '03 Chiller Group_Electric_Active Energy (kWh)',
       '03 Chiller Group_Electric_Active Power (kW)',
       '03 Chiller Group_Electric_Apparent Power (kVa)',
       '03 Chiller Group_Electric_Power Factor (real)',
       '04 UTA_Electric_Active Energy (kWh)',
       '04 UTA_Electric_Active Power (kW)',
       '04 UTA_Electric_Apparent Power (kVa)',
       '04 UTA_Electric_Power Factor (real)',
       '05 Compressors_Electric_Active Energy (kWh)',
       '05 Compressors_Electric_Active Power (kW)',
       '05 Compressors_Electric_Apparent Power (kVa)',
       '05 Compressors_Electric_Power Factor (real)',
       '06 Offices_Electric_Active Energy (kWh)',
       '06 Offices_Electric_Active Power (kW)',
       '06 Offices_Electric_Apparent Power (kVa)',
       '06 

In [6]:
data_center_target = (df['07 Data Center_Electric_Active Energy (kWh)'])
data_center_target.to_csv('data/target_variables/target_data_center.csv')

## Synthesized dataset containing the following columns:

total_data_center_consumption_kwh = β1*Number_of_active_servers + β2*Number_of_active_network_switches + β3*Number_of_active_hard_drives

	•	Number of active servers
	•	Number of active network switches (PoE)
	•	Number of active network switches (Non-PoE)
	•	Number of active hard drives (HDD)
	•	Number of active solid-state drives (SSD)
	•	Total energy consumption in kWh

In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the CSV file
df = pd.read_csv('data/target_variables/target_data_center.csv')

# Define the power consumption per hour for each component (multiply by 4)
power_consumption_per_hour = {
    'server': 0.05279680365 * 4,  # kWh per hour
    'network_switch_poe': (25 / 1000 * 4, 62.5 / 1000 * 4),  # Convert Wh to kWh per hour
    'network_switch_non_poe': (5 / 1000 * 4, 12.5 / 1000 * 4),  # Convert Wh to kWh per hour
    'hdd': (0.175 / 1000 * 4, 0.2625 / 1000 * 4),  # Convert Wh to kWh per hour
    'ssd': 0.175 / 1000 * 4  # Convert Wh to kWh per hour
}

# List of component names
components = list(power_consumption_per_hour.keys())

# Function to calculate total consumption based on the number of active units
def calculate_consumption(x):
    total_consumption = 0
    for i, component in enumerate(components):
        total_consumption += x[i] * power_consumption_per_hour[component] if isinstance(power_consumption_per_hour[component], float) else x[i] * np.mean(power_consumption_per_hour[component])
    return total_consumption

# Function to minimize: difference between calculated and target consumption
def objective(x, target_consumption):
    return np.abs(calculate_consumption(x) - target_consumption)

# List to store synthesized data
synthesized_data = []

# Optimize for each hour in the dataset
for index, row in df.iterrows():
    target_consumption = row['07 Data Center_Electric_Active Energy (kWh)']
    
    # Initial guess: random integers within a reasonable range
    initial_guess = np.random.randint(1, 10, size=len(components))
    
    # Bounds: each component should be at least 0
    bounds = [(0, None)] * len(components)
    
    # Optimize
    result = minimize(objective, initial_guess, args=(target_consumption,), bounds=bounds, method='L-BFGS-B')
    
    # Round the result to the nearest whole number
    rounded_result = np.round(result.x).astype(int)
    
    # Append to the synthesized data
    synthesized_data.append(rounded_result)

# Create a DataFrame for the synthesized data
synthesized_df = pd.DataFrame(synthesized_data, columns=components)
synthesized_df['total_consumption'] = synthesized_df.apply(lambda row: calculate_consumption(row), axis=1)
synthesized_df['target_consumption'] = df['07 Data Center_Electric_Active Energy (kWh)']

synthesized_df.head()

/var/folders/9k/rnh189dj1_95w5p98sc88vbh0000gn/T/ipykernel_1928/1067477145.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_consumption += x[i] * power_consumption_per_hour[component] if isinstance(power_consumption_per_hour[component], float) else x[i] * np.mean(power_consumption_per_hour[component])


       server  network_switch_poe  network_switch_non_poe  hdd  ssd  \
0           1                   0                       1    6    9   
1           1                   0                       0    0    3   
2           0                   1                       1    2    1   
3           0                   1                       1    5    9   
4           1                   0                       1    3    2   
...       ...                 ...                     ...  ...  ...   
10685       0                   1                       1    0    2   
10686       0                   1                       4    2    5   
10687       0                   1                       3    0    1   
10688       0                   0                       5    6    8   
10689       0                   1                       1    0    0   

       total_consumption  target_consumption  
0               0.257737              0.2575  
1               0.213287              0.2500  
2     

In [17]:
# Save synthesized data to CSV
synthesized_df.to_csv('data/synthesized_data/data_center_synthesized.csv', index=False)